In [1]:
pip install transformers torch tqdm


In [2]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import XLMRobertaModel, XLMRobertaTokenizer
from tqdm import tqdm
import pandas as pd

In [3]:
pip install sentencepiece


Note: you may need to restart the kernel to use updated packages.


In [4]:
# Load the pre-trained XLM-RoBERTa model and tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaModel.from_pretrained('xlm-roberta-base')

C:\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# Set the model to evaluation mode
model.eval()

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=Tru

In [6]:
# Load your dataset (adjust the path accordingly)
df = pd.read_csv('C:\\Users\\Hp\Desktop\\Cleaned_datasets\\preprocessed_daigt_V4_dataset.csv')  # Assuming your dataset has a 'text' column

In [7]:
# Define a custom dataset class for loading essays
class EssayDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')
        return encoding

In [8]:
# Initialize the dataset and DataLoader
batch_size = 100  # Adjust batch size based on your GPU/CPU memory
essay_dataset = EssayDataset(df['cleaned_text'].fillna('').tolist(), tokenizer)
essay_loader = DataLoader(essay_dataset, batch_size=batch_size, shuffle=False)


In [9]:
# Function to generate embeddings for a batch of texts
def get_embeddings(batch):
    input_ids = batch['input_ids'].squeeze(1)  # Extract input_ids
    attention_mask = batch['attention_mask'].squeeze(1)  # Extract attention_mask

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token's embeddings
    
    return embeddings


In [10]:
# List to store all embeddings
all_embeddings = []

In [11]:
# Process each batch
for i, batch in enumerate(tqdm(essay_loader, desc="Processing batches")):
    print(f"Processing batch {i + 1} out of {len(essay_loader)}")
    
    # Generate embeddings for the current batch
    batch_embeddings = get_embeddings(batch)
    
    # Collect the embeddings into the list
    all_embeddings.extend(batch_embeddings.cpu().numpy().tolist())  # Ensure to move embeddings to CPU before converting to numpy


Processing batches:   0%|                                                                      | 0/736 [00:00<?, ?it/s]

Processing batch 1 out of 736


Processing batches:   0%|                                                          | 1/736 [04:14<51:53:51, 254.19s/it]

Processing batch 2 out of 736


Processing batches:   0%|▏                                                         | 2/736 [08:45<53:51:13, 264.13s/it]

...

Processing batches: 100%|██████████████████████████████████████████████████████▊| 734/736 [192:29:39<05:11, 155.52s/it]

Processing batch 735 out of 736


Processing batches: 100%|██████████████████████████████████████████████████████▉| 735/736 [192:32:14<02:35, 155.35s/it]

Processing batch 736 out of 736


Processing batches: 100%|███████████████████████████████████████████████████████| 736/736 [192:34:01<00:00, 941.90s/it]


In [12]:
# Convert embeddings to a DataFrame
embeddings_df = pd.DataFrame(all_embeddings)

In [13]:
# Optionally, save the embeddings to a CSV file
embeddings_df.to_csv('C:\\Users\\Hp\Desktop\\XLM_embeddings_in_batches.csv', index=False)

print("Embeddings generated and saved to CSV successfully!")

Embeddings generated and saved to CSV successfully!


In [1]:
import pandas as pd
import numpy as np

# Load your existing dataset with essays
essays_df = pd.read_csv('C:\\Users\\Hp\Desktop\\Cleaned_datasets\\preprocessed_daigt_V4_dataset.csv')

# Load your generated embeddings (assuming embeddings are saved in a separate CSV file)
embeddings_df = pd.read_csv('C:\\Users\\Hp\\Desktop\\Cleaned_datasets\\Embeddings_XLM\\XLM_embeddings_in_batches.csv')  # Ensure each row corresponds to the essay

# Assuming each row of embeddings_df contains the embeddings as individual columns (0, 1, 2, ..., 767 for a 768-dimensional embedding)
# Concatenate the embeddings into a single column as list/array
embeddings_df['XLMRoberta_embeddings'] = embeddings_df.apply(lambda row: np.array(row.values), axis=1)

# Add the 'distilbert_embeddings' column to your existing essays DataFrame
# Ensure the two DataFrames (essays_df and embeddings_df) have the same number of rows and align correctly
essays_df['XLMRoberta_embeddings'] = embeddings_df['XLMRoberta_embeddings']

# Save the merged DataFrame to a new CSV file
essays_df.to_csv("C:\\Users\\Hp\Desktop\\preprocessed_daigt_v4_with_XLM_embeddings.csv", index=False)

print("Dataset with embeddings saved successfully.")

Dataset with embeddings saved successfully.
